In [4]:
import pandas as pd
import numpy as np
import os

In [57]:
# Step 1: Load the Excel file
folder_path = "Data\InputData"

all_data = []
for filename in os.listdir(folder_path):
    if filename.endswith(".csv"):
        file_path = os.path.join(folder_path, filename)
        try:
            df = pd.read_csv(file_path)
            all_data.append(df)
        except Exception as e:
            print(f"Error loading {filename}: {e}")  #Good to print errors to console

if all_data:
    combined_df = pd.concat(all_data, ignore_index=True)  


In [59]:
combined_df['Date'] = pd.to_datetime(df['PerformanceStartDate'])

pivot_df = combined_df.pivot_table(
    index=['elevatorunitId', 'elevatorunitnumber', 'Date'],
    columns='ItemFieldId',
    values='Readvalue',
    aggfunc='sum'
).reset_index()

In [60]:
pivot_df.columns = pivot_df.columns.str.strip().str.replace(' ', '_')
pivot_df.head(2)

ItemFieldId,elevatorunitId,elevatorunitnumber,Date,door_operations,door_reversals,front_door_cycles,front_door_reversals,inop11_levelling_errors,inop2_hoistway_door_lock_failures,inop3_elevator_start_delayed,...,rear_door_cycles,rear_door_reversals,run_starts_ero,run_starts_inspection,run_starts_releveling,run_starts_rescue,run_time_releveling,run_time_rescue,slow_door_operations,total_door_cycles
0,15,S0463,2025-04-01 22:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,108.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,15,S0463,2025-04-02 00:36:53,79.0,1.0,NaN,NaN,NaN,37.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,95.0,108.0


In [61]:
pivot_df['total_door_operations'] = pivot_df[['door_operations', 'front_door_cycles', 'rear_door_cycles']].sum(axis=1)
pivot_df['total_door_reversals'] = pivot_df[['door_reversals', 'front_door_reversals', 'rear_door_reversals']].sum(axis=1)    
pivot_df['door_failure_events'] = pivot_df[['inop4_car_door_open_command_failures', 'inop5_door_open_failures']].sum(axis=1)
pivot_df['hoistway_faults'] = pivot_df['inop2_hoistway_door_lock_failures']
pivot_df['safety_chain_issues'] = pivot_df[['inop7_safety_chain_failures_running', 'inop8_safety_chain_failures_idle']].sum(axis=1)
pivot_df['levelling_total_errors'] = pivot_df[['inop11_levelling_errors', 'levelling_errors', 'number_of_non_level_landings']].sum(axis=1)
pivot_df['startup_delays'] = pivot_df['inop3_elevator_start_delayed']
pivot_df['average_run_time'] = pivot_df[['one_floor_run_time', 'run_time_releveling', 'run_time_rescue']].mean(axis=1)
pivot_df['total_run_starts'] = pivot_df[['run_starts_ero', 'run_starts_inspection', 'run_starts_releveling', 'run_starts_rescue']].sum(axis=1)
pivot_df['total_door_cycles'] = pivot_df['total_door_cycles']

In [63]:
# 4. Drop unused or raw columns (after aggregation)
columns_to_drop = [
    'door_operations','front_door_cycles', 'rear_door_cycles',
    'door_reversals', 'front_door_reversals', 'rear_door_reversals',    
    'inop4_car_door_open_command_failures', 'inop5_door_open_failures',
    'inop2_hoistway_door_lock_failures',
    'inop7_safety_chain_failures_running', 'inop8_safety_chain_failures_idle',
    'inop11_levelling_errors', 'levelling_errors', 'number_of_non_level_landings',
    'inop3_elevator_start_delayed',
    'one_floor_run_time', 'run_time_releveling', 'run_time_rescue',
    'run_starts_ero', 'run_starts_inspection', 'run_starts_releveling', 'run_starts_rescue'
]

pivot_df.drop(columns=[col for col in columns_to_drop if col in pivot_df.columns], inplace=True)
pivot_df.head(2)

ItemFieldId,elevatorunitId,elevatorunitnumber,Date,slow_door_operations,total_door_cycles,total_door_operations,total_door_reversals,door_failure_events,hoistway_faults,safety_chain_issues,levelling_total_errors,startup_delays,average_run_time,total_run_starts
0,15,S0463,2025-04-01 22:00:00,NaN,NaN,108.0,1.0,0.0,NaN,0.0,0.0,NaN,NaN,0.0
1,15,S0463,2025-04-02 00:36:53,95.0,108.0,79.0,1.0,79.0,37.0,0.0,0.0,NaN,862500.0,0.0


In [64]:
 # Door reversal rate per cycle
pivot_df['door_reversal_rate'] = pivot_df['total_door_reversals'] / (pivot_df['total_door_operations'] + 1)

# Safety chain failure ratio
pivot_df['safety_chain_issues_ratio'] = pivot_df['safety_chain_issues'] / (pivot_df['total_run_starts'] + 1)

# Ratio of slow operations to total total_operations
pivot_df['slow_door_operations_ratio'] = pivot_df['slow_door_operations'] / (pivot_df['total_door_operations'] + 1)

# Slow door operation flag
pivot_df['is_slow_door'] = (pivot_df['slow_door_operations'] > 5).astype(int)  

pivot_df.fillna(0, inplace=True)

In [66]:
# Final check
print(pivot_df.columns)

Index(['elevatorunitId', 'elevatorunitnumber', 'Date', 'slow_door_operations',
       'total_door_cycles', 'total_door_operations', 'total_door_reversals',
       'door_failure_events', 'hoistway_faults', 'safety_chain_issues',
       'levelling_total_errors', 'startup_delays', 'average_run_time',
       'total_run_starts', 'door_reversal_rate', 'safety_chain_issues_ratio',
       'slow_door_operations_ratio', 'is_slow_door'],
      dtype='object', name='ItemFieldId')


In [ ]:
# Define fault condition (label engineering)
pivot_df['Fault'] = (
(pivot_df['total_door_reversals'] > 100) |
(pivot_df['door_failure_events'] > 2) |
(pivot_df['hoistway_faults'] > 2) |
(pivot_df['safety_chain_issues'] > 2) |

(pivot_df['levelling_total_errors'] > 10) |
(pivot_df['startup_delays'] > 10) |

(pivot_df['slow_door_operations'] > 5)     
).astype(int) 

In [85]:
# Total records
total_records = len(pivot_df)

# Count faults
fault_counts = pivot_df['Fault'].value_counts().sort_index()
fault_percent = (fault_counts / total_records * 100).round(2)

# Prepare the summary table
summary_table = pd.DataFrame({
    'Fault Label': ['Non-Faulty (0)', 'Faulty (1)', 'Total'],
    'Record Count': [
        fault_counts.get(0, 0),
        fault_counts.get(1, 0),
        total_records
    ],
    'Percentage (%)': [
        fault_percent.get(0, 0),
        fault_percent.get(1, 0),
        100.0
    ]
})

# Print results
print("\n--- Fault Summary Table ---")
print(summary_table.to_string(index=False))



--- Fault Summary Table ---
   Fault Label  Record Count  Percentage (%)
Non-Faulty (0)        317598           80.61
    Faulty (1)         76391           19.39
         Total        393989          100.00


In [67]:
# Save pivot_df to an Excel file
output_path = 'Data/pivot_output1.csv'  # Change path as needed
pivot_df.to_csv(output_path, index=True)